# Purpose of analysis
The purpose of this project is to perform data analysis on Soccer database using pandas and sqlite3 to draw conclusions related to the following questions:

1 - What teams improved the most over the time period?<br>
2 - Which players improved the most in penalties?<br>
3 - What team attributes lead to the most victories?<br>
4 - Do rightfoot players have more shot accuracy/power than leftfoot players?<br>
5 - What player attributes are associated with higher overall rating/potential?<br>
6 - What leagues do most of high rating players play?

# MetaData

The database contains 8 tables as follows:

1 - Country <br>
2 - League<br>
3 - Players<br>
4 - Players' Attributes<br>
5 - Teams<br>
6 - Teams' Attributes<br>
7 - Matches<br>
8 - Sqllite sequences<br>

All information about each table can be found in this [Kaggle link](https://www.kaggle.com/datasets/hugomathien/soccer?resource=download)

# Imports

In [2]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Creating Dataframes from Soccer database

In order to import tables from sqllite into dataframes, we must decide which tables are we interested into. Specifically, we are interested in:<br> 

1 - Intersection between teams and team attributes tables.<br> 
2 - Intersection between players and player attributes tables.<br> 
3 - Intersections between teams and matches tables.<br>
4 - Intersection between players and leagues tables.

# Teams table

In [4]:
# establishing sql connection

conn = sqlite3.connect("database.sqlite") 

#reading query into pandas

sql = pd.read_sql_query("SELECT * FROM ")